In [51]:
import pickle 
import enchant

# Multi Token

In [2]:
open_file = open('set_romanHTs.pkl', "rb")
hashtags = pickle.load(open_file)
open_file.close()

In [3]:
len(hashtags)

7861

In [52]:
english_dict = enchant.Dict("en_US")

In [5]:
single_eng = []
rest = []
for elt in hashtags:
    if english_dict.check(elt.lower()):
        single_eng.append(elt)
    else:
        rest.append(elt)

In [6]:
len(single_eng)

166

In [7]:
len(rest)

7695

In [15]:
import csv
outputStr = ""
for elt in rest: 
    outputStr += elt + "\t" + "roman" + "\n"

with open('multi_token_hashtags_with_source.tsv', 'w', newline='') as f_output:
    f_output.write(outputStr)

In [16]:
open_file = open('set_electionHTs.pkl', "rb")
hashtags = pickle.load(open_file)
open_file.close()

In [11]:
len(hashtags)

720348

In [12]:
single_eng = []
rest = []
for elt in hashtags:
    if english_dict.check(elt.lower()):
        single_eng.append(elt)
    else:
        rest.append(elt)

In [13]:
len(single_eng)

40129

In [14]:
len(rest)

680219

In [17]:
import csv
outputStr = ""
for elt in rest: 
    outputStr += elt + "\t" + "election" "\n"

with open('multi_token_hashtags_with_source.tsv', 'a', newline='') as f_output:
    f_output.write(outputStr + "\t" + "roman")

# Non Separable

In [2]:
import pandas as pd
import re 

In [2]:
df = pd.read_csv('multi_token_hashtags_with_source.tsv', header=None, sep='\t')
df.columns = ['Hashtag', 'Source']
df.head()

,Hashtag,Source
0,trudeauisadog,roman
1,HockeyHair,roman
2,DrKJayakumar,roman
3,Episode1373,roman
4,1stResponder,roman


In [3]:
df = df.dropna()
len(df)

1360434

In [46]:
count = 0
for index, row in df.iterrows():
    if row['Hashtag'].find('_') != -1:
        count += 1
count

54820

In [4]:
def camelCase_split(string):
    splitted = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', string)
    splitted_list = [m.group(0) for m in splitted]
    return ' '.join(splitted_list)

In [5]:
camelCase_split('HelloAkshalaBhatnagar')

'Hello Akshala Bhatnagar'

In [6]:
def underscore_split(string):
    splitted_list = string.split('_')
    return ' '.join(splitted_list)

In [7]:
underscore_split('hello_Akshala Bhatnagar')

'hello Akshala Bhatnagar'

In [8]:
not_separable = pd.DataFrame(columns=['Hashtag', 'Source'])

In [9]:
separable = pd.DataFrame(columns=['Hashtag', 'Source'])

In [10]:
not_separable_count = 0
separable_count = 0
for index, row in df.iterrows():
    camel_case_separated = camelCase_split(row['Hashtag'])
    final_separated = underscore_split(camel_case_separated)
    if final_separated == row['Hashtag']:
        not_separable = not_separable.append(row)
        not_separable_count += 1
    else:
        separable.loc[separable_count] = [final_separated, row['Source']]
        separable_count += 1

In [11]:
not_separable.drop_duplicates(keep = False, inplace = True) 
not_separable.head()

,Hashtag,Source
0,trudeauisadog,roman
3,Episode1373,roman
5,Malabon,roman
6,bjppollmanifesto,roman
7,milan,roman


In [12]:
len(not_separable)

667830

In [13]:
separable.drop_duplicates(keep = False, inplace = True) 
separable.head()

,Hashtag,Source
0,Hockey Hair,roman
1,Dr K Jayakumar,roman
2,1st Responder,roman
3,Bharat K,roman
4,Congress,roman


In [14]:
len(separable)

676706

In [15]:
not_separable.to_csv('not_separable_with_source.csv')

In [16]:
separable.to_csv('separable_with_source.csv')

# Non eng

In [36]:
not_seaparable_hashtags = pd.read_csv('not_separable_with_source.csv')

In [37]:
not_seaparable_hashtags = not_separable['Hashtag']
type(not_seaparable_hashtags)

pandas.core.series.Series

In [38]:
not_seaparable_hashtags.to_csv('not_separable_hashtags_with_source.tsv', sep='\t')

In [39]:
df = pd.read_csv('not_separable_hashtags_with_source.tsv', sep='\t')
df.head()

,Unnamed: 0,Hashtag
0,0,trudeauisadog
1,3,Episode1373
2,5,Malabon
3,6,bjppollmanifesto
4,7,milan


In [40]:
df = df.drop('Unnamed: 0', axis=1)
df.head()

,Hashtag
0,trudeauisadog
1,Episode1373
2,Malabon
3,bjppollmanifesto
4,milan


In [41]:
df.to_csv('not_separable_hashtags_with_source.tsv', sep='\t', header=False, index=False)

In [42]:
!python hashtag_master-master/word_breaker/main.py --k 10 --lm hashtag_master-master/data/small_gt.bin --out not_separable_hashtags_with_source_topk.tsv --input not_separable_hashtags_with_source.tsv

Loading language model.
Done.
Extracting top segmentations.


In [48]:
df = pd.read_csv('not_separable_hashtags_with_source_topk.tsv', sep='\t', header=None, names=['Hashtag', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
df.head()

,Hashtag,1,2,3,4,5,6,7,8,9,10
0,trudeauisadog,trudeau is a dog,trudeau isa dog,t rudeau is a dog,trudeau i s a dog,trudeau i sa dog,trudeau is a d o g,trudeau is a d og,trudeau i sad og,tru de au is a dog,trudeau is a do g
1,Episode1373,Episode 13 73,Episode 1 373,Episode 137 3,Episode1 373,Episode 1 37 3,Episode 13 7 3,Episode 1 3 73,Episode1 37 3,Episode1 3 73,Episode 1 3 7 3
2,Malabon,Malab on,Malabo n,Mala bon,Ma lab on,Mala b on,Mal a b on,M a lab on,Mal abon,Mala bo n,Mal ab on
3,bjppollmanifesto,bjp poll manifesto,bj p poll manifesto,b jp poll manifesto,bjp poll manifest o,bj p p o l l manifesto,b j p poll manifesto,bjp p o l l manifesto,bjp po ll manifesto,bjp pol l manifesto,bj p poll manifest o
4,milan,milan,mil an,mila n,mi lan,m i l a n,mil a n,mi la n,mi l a n,m i l an,m ilan


In [49]:
non_eng_df = pd.DataFrame(columns=['Hashtag', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])

In [53]:
for index, row in df.iterrows():
    try:
        segmented = row['1'].split()
        flag = 0
        for elt in segmented:
            if not elt.isnumeric() and not english_dict.check(elt.lower()):
                flag = 1
        if flag:
            non_eng_df = non_eng_df.append(row)
    except AttributeError:
        pass

In [54]:
non_eng_df.head()

,Hashtag,1,2,3,4,5,6,7,8,9,10
0,trudeauisadog,trudeau is a dog,trudeau isa dog,t rudeau is a dog,trudeau i s a dog,trudeau i sa dog,trudeau is a d o g,trudeau is a d og,trudeau i sad og,tru de au is a dog,trudeau is a do g
2,Malabon,Malab on,Malabo n,Mala bon,Ma lab on,Mala b on,Mal a b on,M a lab on,Mal abon,Mala bo n,Mal ab on
3,bjppollmanifesto,bjp poll manifesto,bj p poll manifesto,b jp poll manifesto,bjp poll manifest o,bj p p o l l manifesto,b j p poll manifesto,bjp p o l l manifesto,bjp po ll manifesto,bjp pol l manifesto,bj p poll manifest o
4,milan,milan,mil an,mila n,mi lan,m i l a n,mil a n,mi la n,mi l a n,m i l an,m ilan
5,Dagga,Dagga,Dagg a,Dag ga,Da gga,Da gg a,Da g ga,Dag g a,D agg a,Da g g a,D a gga


In [55]:
len(non_eng_df)

450614

In [57]:
non_eng_df.to_csv('nonEng_notSeparable_multiToken_withSource.csv')

# Final Segmentation

In [58]:
non_eng_df = pd.read_csv('nonEng_notSeparable_multiToken_withSource.csv')

In [59]:
nonEng_hashtags = non_eng_df['Hashtag']
type(nonEng_hashtags)

pandas.core.series.Series

In [60]:
nonEng_hashtags.to_csv('nonEng_notSeparable_multiToken_hashtags_withSource.tsv', sep='\t')

In [61]:
df = pd.read_csv('nonEng_notSeparable_multiToken_hashtags_withSource.tsv', sep='\t')
df.head()

,Unnamed: 0,Hashtag
0,0,trudeauisadog
1,1,Malabon
2,2,bjppollmanifesto
3,3,milan
4,4,Dagga


In [62]:
df = df.drop('Unnamed: 0', axis=1)
df.head()

,Hashtag
0,trudeauisadog
1,Malabon
2,bjppollmanifesto
3,milan
4,Dagga


In [63]:
df.to_csv('nonEng_notSeparable_multiToken_hashtags_withSource.tsv', sep='\t', header=False, index=False)

In [71]:
!python hashtag_master-master/word_breaker/main.py --k 10 --lm bigger_model.binary --out nonEng_notSeparable_multiToken_hashtags_withSource_topk.tsv --input nonEng_notSeparable_multiToken_hashtags_withSource.tsv

Loading language model.
Done.
Extracting top segmentations.


In [3]:
df = pd.read_csv('nonEng_notSeparable_multiToken_hashtags_withSource_topk.tsv', sep='\t', header=None, names=['Hashtag', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
df.head()

,Hashtag,1,2,3,4,5,6,7,8,9,10
0,trudeauisadog,trudeauisadog,trudeau is a dog,trudeauisado g,trudeauisa dog,t rudeauisadog,trudeauisad og,tr udeauisadog,trudeau isadog,tru deauisadog,trude auisadog
1,Malabon,Malabon,Malab on,Mala bon,Ma labon,Malabo n,Ma lab on,Mala b on,Mal abon,Mal ab on,Mal a bon
2,bjppollmanifesto,bjppollmanifesto,bjppollmanifes to,bjppollmanifest o,b jppollmanifesto,bjppoll manifesto,bjp pollmanifesto,bj ppollmanifesto,bjppollmanife sto,bjppollmanif esto,bjppollmanife s to
3,milan,milan,mi lan,mil an,mila n,m ilan,m i l a n,mil a n,mi la n,m i lan,m il an
4,Dagga,Dagga,Dagg a,Dag ga,Da gga,D agga,Dag g a,Da g g a,Da gg a,Da g ga,D ag ga


In [4]:
df_source = pd.read_csv('multi_token_hashtags_with_source.tsv', header=None, sep='\t')
df_source.columns = ['Hashtag', 'Source']
df_source.head()

,Hashtag,Source
0,trudeauisadog,roman
1,HockeyHair,roman
2,DrKJayakumar,roman
3,Episode1373,roman
4,1stResponder,roman


In [5]:
df_final = pd.merge(df, df_source, on=['Hashtag'])
df_final.head()

,Hashtag,1,2,3,4,5,6,7,8,9,10,Source
0,trudeauisadog,trudeauisadog,trudeau is a dog,trudeauisado g,trudeauisa dog,t rudeauisadog,trudeauisad og,tr udeauisadog,trudeau isadog,tru deauisadog,trude auisadog,roman
1,trudeauisadog,trudeauisadog,trudeau is a dog,trudeauisado g,trudeauisa dog,t rudeauisadog,trudeauisad og,tr udeauisadog,trudeau isadog,tru deauisadog,trude auisadog,election
2,trudeauisadog,trudeauisadog,trudeau is a dog,trudeauisado g,trudeauisa dog,t rudeauisadog,trudeauisad og,tr udeauisadog,trudeau isadog,tru deauisadog,trude auisadog,roman
3,trudeauisadog,trudeauisadog,trudeau is a dog,trudeauisado g,trudeauisa dog,t rudeauisadog,trudeauisad og,tr udeauisadog,trudeau isadog,tru deauisadog,trude auisadog,election
4,Malabon,Malabon,Malab on,Mala bon,Ma labon,Malabo n,Ma lab on,Mala b on,Mal abon,Mal ab on,Mal a bon,roman


In [6]:
len(df_final)

901228

In [7]:
df_final.to_csv('finalSegmented_withSource.csv')